In [4]:
import os
import openai
import langchain
from pinecone import Pinecone
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings.openai import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
from langchain_community.llms import OpenAI
from typing import List
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
try:
    from langchain_core.documents import Document
except ImportError:
    from langchain.schema import Document

def read_doc(directory: str) -> List[Document]:
    loader = PyPDFDirectoryLoader(directory)
    documents = loader.load()
    return documents

In [6]:
docs = read_doc("documents/")
print(f"Loaded {len(docs)} documents.")

Loaded 60 documents.


In [7]:
docs[0]

Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': 'documents\\budget_speech.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025')

In [8]:
def chunk_data(docs: List[Document], chunk_size: int = 800, chunk_overlap: int = 200) -> List[Document]:
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
    )

    doc = text_splitter.split_documents(docs)
    return doc

In [9]:
documents = chunk_data(docs)
print(f"Chunked into {len(documents)} documents.")
#documents

Chunked into 158 documents.


In [10]:
documents[0]

Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-02-01T03:54:26+05:30', 'author': 'hss', 'moddate': '2025-02-01T03:56:01+05:30', 'title': '', 'source': 'documents\\budget_speech.pdf', 'total_pages': 60, 'page': 0, 'page_label': '1'}, page_content='GOVERNMENT OF INDIA\nBUDGET 2025-2026\nSPEECH\nOF\nNIRMALA SITHARAMAN\nMINISTER OF FINANCE\nFebruary 1,  2025')

In [11]:
embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))

C:\Users\OlavoDefendiDalberto\AppData\Local\Temp\ipykernel_11716\3059526767.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-openai package and should be used instead. To use it run `pip install -U `langchain-openai` and import as `from `langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(api_key=os.getenv("OPENAI_API_KEY"))


In [12]:
vectors = embeddings.embed_query("How are you?")
len(vectors)

1536

In [13]:
index_name = os.getenv("PINECONE_INDEX_NAME")
docsearch = PineconeVectorStore.from_documents(
    documents=documents, 
    embedding=embeddings, 
    index_name=index_name
)
print("Documentos indexados com sucesso!")

Documentos indexados com sucesso!


In [ ]:
def retrieve_query(query, k=2):
    matching_results = docsearch.similarity_search(query, k=k)
    return matching_results

In [14]:
from langchain.chains.question_answering import load_qa_chain
from langchain import OpenAI

ModuleNotFoundError: No module named 'langchain.chains'